This script builds a dataset of prompts for programming languages that do not perform very well. It adds a "Generate for LANGUAGE" prefix to each prompt. Hopefully that will help!

You may wish to change the paths below.

In [1]:
%set_env TRANSFORMERS_CACHE=/work/arjunguha-research-group/arjun/cache/transformers
%set_env HF_DATASETS_CACHE=/work/arjunguha-research-group/arjun/cache/datasets

env: TRANSFORMERS_CACHE=/work/arjunguha-research-group/arjun/cache/transformers
env: HF_DATASETS_CACHE=/work/arjunguha-research-group/arjun/cache/datasets


In [28]:
import datasets
import json
from pathlib import Path

In [48]:
LANGS = [ "jl", "rkt", "d", "r" ]
TARGET_PATH = Path("../language_in_prompt")

First, we concatenate the datasets for some languages. I regret that these are subsets.

We are careful to insert the `;; Generate in Racket` after the `#lang racket`.
This explains the magic number that appears in the cell after.

In [46]:
dataset = datasets.load_dataset("nuprl/MultiPL-E", name=f"humaneval-r", split="test")
print(dataset[0]["prompt"])


Found cached dataset multi_pl-e (/work/arjunguha-research-group/arjun/cache/datasets/nuprl___multi_pl-e/humaneval-r/2.0.0/cd5d4cbadd8c38aeada4809afb908ef4425f83de6f2726bf2639f6980002d4c0)


# For a given number n, find the largest number that divides n evenly, smaller than n
# >>> largest_divisor(15)
# 5
largest_divisor <- function(n) {


In [47]:
#print(dataset[250]["prompt"][:14] + "<SPLIT>" + dataset[250]["prompt"][14:])

def add_language_name_to_prompt(item):
    lang = item["language"]
    prompt = item["prompt"]
    if lang == "jl":
        prompt = "# Generate in Julia\n\n" + prompt
    elif lang == "rkt":
        prompt = "#lang racket\n\n ;; Generate in Racket\n\n" + prompt[14:]
    elif lang == "d":
        prompt = "/* Generate in D */\n" + prompt
    elif lang == "r":
        prompt = "# Generate in R\n\n" + prompt
    else:
        raise ValueError(f"Unknown language {lang}")
    return { "prompt": prompt, "prompt_terminology": "promptwithlang" }

In [49]:
for lang in LANGS:
    dataset = datasets.load_dataset("nuprl/MultiPL-E", name=f"humaneval-{lang}", split="test")
    dataset = dataset.map(add_language_name_to_prompt)
    dataset.to_json(TARGET_PATH / f"humaneval-{lang}.json")

Found cached dataset multi_pl-e (/work/arjunguha-research-group/arjun/cache/datasets/nuprl___multi_pl-e/humaneval-jl/2.0.0/cd5d4cbadd8c38aeada4809afb908ef4425f83de6f2726bf2639f6980002d4c0)
Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 30.59ba/s]
Found cached dataset multi_pl-e (/work/arjunguha-research-group/arjun/cache/datasets/nuprl___multi_pl-e/humaneval-rkt/2.0.0/cd5d4cbadd8c38aeada4809afb908ef4425f83de6f2726bf2639f6980002d4c0)
Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 156.41ba/s]
Found cached dataset multi_pl-e (/work/arjunguha-research-group/arjun/cache/datasets/nuprl___multi_pl-e/humaneval-d/2.0.0/cd5d4cbadd8c38aeada4809afb908ef4425f83de6f2726bf2639f6980002d4c0)
Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 156.88ba/s]
Found cached dataset multi_pl-e (/work/arjunguha-research-group/arjun/cache/datasets/nuprl___multi_pl-e/humaneval-r/2.0.0/cd5d4cbadd8c38aeada4809afb908ef4425f83de6f2726bf2639f6980002d4c0)
Creati